<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 166 kB 64.7 MB/s 
     |████████████████████████████████| 182 kB 73.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 69.3 MB/s 
     |████████████████████████████████| 162 kB 74.2 MB/s 
     |████████████████████████████████| 162 kB 76.7 MB/s 
     |████████████████████████████████| 158 kB 61.2 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 75.1 MB/s 
     |████████████████████████████████| 157 kB 78.1 MB/s 
     |████████████████████████████████| 157 kB 64.1 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 74.8 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 156 kB 79.9 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [12]:
mobilenet = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_mobilenet=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  mobilenet,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
mobilenet.trainable=False
model_mobilenet.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
model_mobilenet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [14]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "MobilenetV2",
)

In [15]:
%%time
epochs=40
history = model_mobilenet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 0.9474 - accuracy: 0.6662

wandb: Adding directory to artifact (/content/wandb/run-20221106_153052-10ddmglg/files/model-best)... Done. 0.1s


22/22 [==============================] - 25s 1s/step - loss: 0.9474 - accuracy: 0.6662 - val_loss: 0.5040 - val_accuracy: 0.8343
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.3458 - accuracy: 0.8892

wandb: Adding directory to artifact (/content/wandb/run-20221106_153052-10ddmglg/files/model-best)... Done. 0.1s


22/22 [==============================] - 21s 972ms/step - loss: 0.3458 - accuracy: 0.8892 - val_loss: 0.2709 - val_accuracy: 0.9371
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 0.2112 - accuracy: 0.9389

...

22/22 [==============================] - 21s 975ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0594 - val_accuracy: 0.9714
Epoch 36/40
22/22 [==============================] - ETA: 0s - loss: 0.0074 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221106_153052-10ddmglg/files/model-best)... Done. 0.1s


22/22 [==============================] - 21s 968ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0589 - val_accuracy: 0.9714
Epoch 37/40
22/22 [==============================] - ETA: 0s - loss: 0.0078 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221106_153052-10ddmglg/files/model-best)... Done. 0.1s


22/22 [==============================] - 21s 961ms/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.0551 - val_accuracy: 0.9771
Epoch 38/40
22/22 [==============================] - 3s 99ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.0597 - val_accuracy: 0.9714
Epoch 39/40
22/22 [==============================] - 3s 100ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0584 - val_accuracy: 0.9714
Epoch 40/40
22/22 [==============================] - 3s 98ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0577 - val_accuracy: 0.9771
CPU times: user 9min 31s, sys: 31.7 s, total: 10min 3s
Wall time: 15min 9s


In [16]:
wandb.finish()

accuracy,▁▆▇▇████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▇▆▇▆▇▇▇▇▇▇▇▇▇██████████▇▇████▇████████
val_loss,█▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,36
best_val_loss,0.05513
epoch,39
loss,0.00648
val_accuracy,0.97714


In [11]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
